# Build Chatbot
`11_chatbot.ipynb`
- https://python.langchain.com/docs/tutorials/chatbot/

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi!, I am bob.'),
    AIMessage(content='Hello bob. how can I help you.'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builider
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [ ]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}}  # 채팅방 아이디 (바뀌면 다른 대화가 된다.)

graph.invoke({'messages': messages}, config=config)

In [ ]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}}  # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='say my name.')
]
graph.invoke({'messages': messages}, config=config)

## Langgraph + `PromptTemplate`

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐.' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


## State 확장

In [34]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [35]:
config = {'configurable': {'thread_id': 'abc1'}} 
state = {
    'messages': [HumanMessage(content='Hi. I am bob')],
    'lang': 'Spanish'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

Hi. I am bob
================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿En qué puedo ayudarte hoy?
